# Generate cross-correlation function between 2 given catalogs

In [1]:
import os
import tempfile
import time
import copy

import numpy as np
import scipy
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (10, 8)

from pypower import CatalogMesh, MeshFFTPower, CatalogFFTPower, PowerSpectrumStatistics, utils, setup_logging
from pycorr import TwoPointCorrelationFunction, TwoPointEstimator, NaturalTwoPointEstimator, project_to_multipoles, BoxSubsampler, project_to_wp, utils, setup_logging
from abacusnbody.data.compaso_halo_catalog import CompaSOHaloCatalog
from cosmoprimo import *

# To activate logging
setup_logging()

### Output directory

In [2]:
output_dir = '/feynman/work/dphp/mp270220/outputs/'

### Get halo catalog

In [3]:
# 1 Gpc/h
path_to_sim_1 = '/feynman/scratch/dphp/ar264273/Abacus/AbacusSummit_highbase_c000_ph100/halos/z1.175'

# 2 Gpc/h
path_to_sim_2_z1 = '/feynman/scratch/dphp/ar264273/Abacus/AbacusSummit_base_c000_ph000/halos/z1.175'
path_to_sim_2_z2 = '/feynman/scratch/dphp/ar264273/Abacus/AbacusSummit_base_c000_ph000/halos/z0.800'

path_to_sim = [path_to_sim_1,
               path_to_sim_2_z1,
               path_to_sim_2_z2]

catalog_names = ['AbacusSummit_highbase_c000_ph100_z1.175',
                 'AbacusSummit_base_c000_ph000_z1.175',
                 'AbacusSummit_base_c000_ph000_z0.800']

sim_z = [1.175, 1.175, 0.800]
sim_boxsizes = [1000, 2000, 2000]

# Simulation to use
sim_index = 0

# Whether to apply RSD or not
RSD = False
RSD_string = ''
if RSD:
    RSD_string = '_wRSD'

In [4]:
boxsize = sim_boxsizes[sim_index]
z = sim_z[sim_index]

In [5]:
# density split names
subset_names = ['D1', 'D2']
split_catalogs_positions = list()

for i in range(len(subset_names)):
    split_catalogs_positions.append(np.load(output_dir+catalog_names[sim_index]+RSD_string+subset_names[i]+'_positions.npy'))

In [6]:
catalog_positions = np.load(output_dir+catalog_names[sim_index]+RSD_string+'_positions.npy')
data_size = np.shape(catalog_positions)[1]

### Generate random catalogs

In [7]:
# Generate random catalogs to compare catalog with when computing correlation function
def generate_catalogs(size=10000, boxsize=(boxsize,)*3, offset=(boxsize,0.,0.), seed=42):
    rng = np.random.RandomState(seed=seed)
    positions = [o + rng.uniform(0., 1., size)*b for o, b in zip(offset, boxsize)]
    return positions

split_randoms_positions = list()

for i in range(len(subset_names)):
    split_randoms_positions.append(generate_catalogs(size=6*data_size, boxsize=(boxsize,)*3, offset=(-boxsize/2.,-boxsize/2.,-boxsize/2.)))
    
catalog_randoms_positions = generate_catalogs(size=6*data_size, boxsize=(boxsize,)*3, offset=(-boxsize/2.,-boxsize/2.,-boxsize/2.))

### Split in subsamples for jackknife variance estimation

In [10]:
subsamplers = list()
labels = list()
split_samples = list()
catalog_samples = list()
split_randoms_samples = list()
catalog_randoms_samples = list()

for i in range(len(subset_names)):
    subsampler = BoxSubsampler(positions=split_catalogs_positions[i], boxsize=np.full(3, boxsize), boxcenter=np.full(3, 0), nsamples=1e3)
    subsamplers.append(subsampler)
    labels.append(subsampler.label(positions=split_catalogs_positions[i]))
    split_samples.append(subsampler.label(split_catalogs_positions[i]))
    split_randoms_samples.append(subsampler.label(split_randoms_positions[i]))
    
subsampler = BoxSubsampler(positions=catalog_positions, boxsize=np.full(3, boxsize), boxcenter=np.full(3, 0), nsamples=1e3)
subsamplers.append(subsampler)
labels.append(subsampler.label(positions=catalog_positions))
catalog_samples.append(subsampler.label(catalog_positions))
catalog_randoms_samples.append(subsampler.label(catalog_randoms_positions))

## Compute s-mu two point correlation function

### Correlation function of each density split

In [11]:
edges = (np.linspace(0., 150., 51), np.linspace(-1, 1, 201))

results = list()

for i in range(len(subset_names)):
    result = TwoPointCorrelationFunction('smu', edges, 
                                         data_positions1=split_catalogs_positions[i], randoms_positions1=split_randoms_positions[i],
                                         data_samples1=split_samples[i], randoms_samples1=split_randoms_samples[i],
                                         data_positions2=catalog_positions, randoms_positions2=catalog_randoms_positions,
                                         data_samples2=catalog_samples, randoms_samples2=catalog_randoms_samples,
                                         engine='corrfunc', nthreads=128,
                                         los = 'x')
    results.append(result)

    # save result
    result.save(output_dir+catalog_names[sim_index]+RSD_string+'_randomx6_CCF_galaxySplit_galaxy'+subset_names[i])

[000031.21]  04-13 17:07  TwoPointCorrelationFunction  INFO     Using estimator <class 'pycorr.twopoint_jackknife.JackknifeLandySzalayTwoPointEstimator'>.
[000031.21]  04-13 17:07  TwoPointCorrelationFunction  INFO     Computing two-point counts D1D2.


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed